In [1]:
import csv,sys
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_curve

In [2]:
def Calc_equal_err_rate(realUser_scores,fakeUser_scores):
    labels = [0]*len(realUser_scores) + [1]*len(fakeUser_scores)
    fpr, tpr, thresholds = roc_curve(labels, realUser_scores + fakeUser_scores)
    # false alarm rate = fpr (false positive rate) 
    false_alarm_rates = fpr
    
    # hitrate = tpr (true positive rate)
    miss_rate= 1 - tpr
    all_dist = miss_rate - false_alarm_rates
    
    idx1 = np.argmin(all_dist[all_dist >=0 ])
    idx2 = np.argmax(all_dist[all_dist < 0])
    
    x_ptr = [miss_rate[idx1], false_alarm_rates[idx1]]
    y_ptr = [miss_rate[idx2], false_alarm_rates[idx2]]
    m = ( x_ptr[0] -x_ptr[1] ) / ( y_ptr[1] - x_ptr[1] - y_ptr[0] + x_ptr[0] )
   
    equal_err_rate = x_ptr[0] + m * ( y_ptr[0] - x_ptr[0] )
#     print (round(equal_err_rate,4))
    
    return equal_err_rate
    

In [3]:
def Manhattan_Scaled(person):
    realUser_scores = []                         # real user score
    fakeUser_scores = []                         # imposter user score
    
    # Consider current person as real and rest as fake
    realUser_data = data.loc[data.subject == person, "H.period":"H.Return"]
    
    fakeUser_data = data.loc[data.subject != person, :]
    
    # genuine user's first 200 time vectors for training
    real_train = realUser_data[:200]
    
    # genuine user's rest 200 time vectors for test 
    # True set (200 records)
    real_test = realUser_data[200:]
    
    # False set (250 records, 5 per fake, 50 fake in all)
    fake_test = fakeUser_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]

#     training...
    mean = real_train.mean().values
    mad = real_train.mad().values    #mean absolute deviaition 
  
    
#     testing...
    rows= real_test.shape[0]
    for r in range(rows):
        score =0 
        for m in range(len(mean)):
            score=score + abs(real_test.iloc[r].values[m] -mean[m] )/mad[m]
        realUser_scores.append(score)

    rows= fake_test.shape[0]
    for r in range(rows):
        score =0 
        for m in range(len(mean)):
            score=score + abs(fake_test.iloc[r].values[m] -mean[m])/mad[m]
        fakeUser_scores.append(score)
    return Calc_equal_err_rate(realUser_scores,fakeUser_scores)

In [4]:
def main():
    global data
    data = pd.read_csv("./dataset/DSL-StrongPasswordData.csv")
    subjects = data["subject"].unique()    # 51 total
    header = [c for c in data]
    equal_err_rate =[]
    for person in subjects:
        equal_err_rate.append(Manhattan_Scaled(person))

    print (np.mean(equal_err_rate), np.std(equal_err_rate))

In [5]:
if __name__ == main():
    main()

0.0945442390728 0.0683754355337
